In [1]:
import sys
sys.path.append("../..") 

In [2]:
import pandas as pd
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.metrics.metrics import gaussian_eval_fn

In [3]:
def g(x):
    x = x.copy()
    x = np.log(x) - np.log(140)
    return x
def ginv(x):
    x = x.copy()
    x = np.exp(x + np.log(140))
    return x


In [20]:
df = pd.read_csv("../../data/simulation_5000_v0.csv")

In [23]:
df = df.loc[df.obs == True,:]
print(sum(df.insulin_t > 0))
print(df.shape)

17480
(38948, 13)


In [6]:
def get_all_predictions(path):
    df_predictions = []
    print(os.listdir(path))
    for folder in os.listdir(path):
        if folder.startswith('.') == False:
            dfs = []
            files = glob.glob(os.path.join(path, folder, "predictions*"))
            for f in files:
                pred = pd.read_csv(f)
                n = re.findall(r'\d+', f)[-2]
                folder_n = re.findall(r'\d+', folder)
                pred.loc[:,'folder'] = int(folder_n[0])
                dfs.append(pred)
            df = pd.concat(dfs, axis=0)
            df_predictions.append(df)
    df_prediction = pd.concat(df_predictions, axis=0)
    return df_prediction

In [7]:
df1 = get_all_predictions("../../experiments/mimic/data_size_5000/")

['version_1', 'version_2', 'version_28', 'version_13', 'version_18', 'version_7', 'version_26', 'version_11', 'version_12', 'version_22', 'version_25', 'version_23', 'version_4', 'version_6', 'version_3', 'version_16', 'version_20', 'version_0', 'version_21', 'version_9', 'version_10', 'version_29', 'version_19', 'version_15', 'version_24', 'version_17', 'version_8', 'version_5', 'version_27', 'version_14']


In [8]:
df1 = df1.loc[df1.folder <= 9,:]

In [9]:
df1

,rn,mu,sigma,model,folder
0,723,-0.004207,0.032304,Catboost,1
1,752,0.004073,0.029290,Catboost,1
2,802,-0.055830,0.033584,Catboost,1
3,854,-0.052387,0.020622,Catboost,1
4,897,-0.049785,0.025956,Catboost,1
...,...,...,...,...,...
6550,1445348,-0.000162,0.016807,ODELSTMModel,5
6551,1445378,0.070781,0.021995,ODELSTMModel,5
6552,1445417,0.041624,0.031421,ODELSTMModel,5
6553,1445449,0.012524,0.024276,ODELSTMModel,5


In [10]:
df_results = pd.merge(df1,df,how = 'left',on='rn')

In [11]:
df_results

,rn,mu,sigma,model,folder,t,glucose_t,glucose_t_obs,obs,insulin_t,dextrose_t,id,glucose_t_obs_next,t1,n,msk,msk0
0,723,-0.004207,0.032304,Catboost,1,0.0,151.882683,151.882683,True,0.05,0.000000,3,139.830080,2.9,1.0,0.0,0.0
1,752,0.004073,0.029290,Catboost,1,2.9,139.830080,139.830080,True,0.00,0.000000,3,126.522468,7.9,2.0,0.0,0.0
2,802,-0.055830,0.033584,Catboost,1,7.9,126.522468,126.522468,True,0.00,0.000000,3,131.852106,13.1,3.0,0.0,0.0
3,854,-0.052387,0.020622,Catboost,1,13.1,131.852106,131.852106,True,0.00,0.000000,3,133.067380,17.4,4.0,0.0,0.0
4,897,-0.049785,0.025956,Catboost,1,17.4,133.067380,133.067380,True,0.00,0.000000,3,133.120141,22.8,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65545,1445348,-0.000162,0.016807,ODELSTMModel,5,7.1,140.728667,140.728667,True,0.05,0.000000,5997,138.470850,10.1,3.0,0.0,0.0
65546,1445378,0.070781,0.021995,ODELSTMModel,5,10.1,138.470850,138.470850,True,0.00,3.714442,5997,148.610889,14.0,4.0,0.0,0.0
65547,1445417,0.041624,0.031421,ODELSTMModel,5,14.0,148.610889,148.610889,True,0.05,3.752021,5997,148.546228,17.2,5.0,0.0,0.0
65548,1445449,0.012524,0.024276,ODELSTMModel,5,17.2,148.546228,148.546228,True,0.05,0.000000,5997,149.442478,20.0,6.0,0.0,0.0


In [12]:
del df
del df1

In [13]:
df_results.loc[:,'target'] = g(df_results.glucose_t_obs)

In [14]:
def gaussian_eval_fn_wrapper(x):
    res = gaussian_eval_fn(x.loc[:,['mu','sigma']].to_numpy(),y=x.target,ginv=ginv)
    return(pd.Series(res))

In [15]:
df_results_mf = df_results.loc[df_results.msk == 0,['model','target','mu','sigma']].dropna().groupby(['model'])[['target','mu','sigma']]
df_metrics = df_results_mf.apply(gaussian_eval_fn_wrapper)

In [16]:
df_metrics

,crps_mean,ig_mean,int_score_mean,var_pit,int_coverage,int_av_width,int_med_width,rmse
model,,,,,,,,
Catboost,0.034525,0.342410,94.774642,0.114655,0.830511,14.297487,13.130895,10.916752
DecayGRUModel,0.034218,1.318921,66.099676,0.099420,0.849428,19.642823,17.866550,10.545667
FlowGRUModel,0.033731,1.475071,48.761596,0.132566,0.806407,16.180029,12.650730,10.039133
FlowLSTMModel,0.034677,0.932888,85.486000,0.122456,0.822426,14.898178,12.722143,10.722450
GRUModel,0.032121,1.290723,61.585556,0.120655,0.808238,15.603177,13.261510,9.654278
IMODE,0.031056,1.274987,82.360199,0.078641,0.883295,17.133791,14.856080,10.696902
LSTMModel,0.039547,0.598745,97.746161,0.115351,0.794203,18.328206,17.209825,11.399290
LinearModel,0.041815,1.060172,70.848438,0.036259,0.969031,58.599343,58.501349,10.588121
ODEGRUModel,0.030749,1.539952,59.951478,0.103174,0.840427,15.571725,12.439449,10.320695


In [17]:
df_results_mf = df_results.loc[(df_results.msk == 0) & (df_results.insulin_t > 0),['model','target','mu','sigma']].dropna().groupby(['model'])[['target','mu','sigma']]
df_metrics = df_results_mf.apply(gaussian_eval_fn_wrapper)
df_metrics

,crps_mean,ig_mean,int_score_mean,var_pit,int_coverage,int_av_width,int_med_width,rmse
model,,,,,,,,
Catboost,0.036863,0.091086,114.654013,0.030622,0.818997,13.668963,12.548348,12.685843
DecayGRUModel,0.034704,1.585537,62.279528,0.025103,0.830289,19.050483,16.553384,11.997193
FlowGRUModel,0.028279,1.783365,30.726651,0.040899,0.843241,15.929793,11.402268,9.887689
FlowLSTMModel,0.036612,1.006554,101.665526,0.041008,0.797077,14.218274,12.368336,12.381107
GRUModel,0.030134,1.537898,55.356417,0.047393,0.821654,15.926930,12.904929,10.315760
IMODE,0.034189,1.320289,101.070570,0.036367,0.865493,17.097248,14.959818,12.713984
LSTMModel,0.040167,0.846095,106.707295,0.027096,0.802391,18.478369,17.042359,12.837597
LinearModel,0.041599,1.096348,67.943646,0.017482,0.975756,59.357301,59.075618,11.428339
ODEGRUModel,0.030495,1.841043,50.846553,0.034754,0.840252,15.584567,11.775797,11.752849


In [18]:
df_results_mf = df_results.loc[(df_results.msk == 0) & (df_results.insulin_t > 0),['model','target','mu','sigma']].dropna().groupby(['model'])[['target','mu','sigma']]
df_metrics = df_results_mf.apply(gaussian_eval_fn_wrapper)
df_metrics

,crps_mean,ig_mean,int_score_mean,var_pit,int_coverage,int_av_width,int_med_width,rmse
model,,,,,,,,
Catboost,0.036863,0.091086,114.654013,0.030622,0.818997,13.668963,12.548348,12.685843
DecayGRUModel,0.034704,1.585537,62.279528,0.025103,0.830289,19.050483,16.553384,11.997193
FlowGRUModel,0.028279,1.783365,30.726651,0.040899,0.843241,15.929793,11.402268,9.887689
FlowLSTMModel,0.036612,1.006554,101.665526,0.041008,0.797077,14.218274,12.368336,12.381107
GRUModel,0.030134,1.537898,55.356417,0.047393,0.821654,15.926930,12.904929,10.315760
IMODE,0.034189,1.320289,101.070570,0.036367,0.865493,17.097248,14.959818,12.713984
LSTMModel,0.040167,0.846095,106.707295,0.027096,0.802391,18.478369,17.042359,12.837597
LinearModel,0.041599,1.096348,67.943646,0.017482,0.975756,59.357301,59.075618,11.428339
ODEGRUModel,0.030495,1.841043,50.846553,0.034754,0.840252,15.584567,11.775797,11.752849
